# Get Nightlight Data from Google Earth Engine using GEEMap

In [ ]:
import ee
import geemap

## Authenticate & Initialize GEE

Requires a [Google Cloud Project](https://console.cloud.google.com/projectcreate) and to enable the [Earth Engine API](https://console.cloud.google.com/apis/api/earthengine.googleapis.com) for the project. Find detailed instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication).

In [ ]:
ee.Authenticate()
# errors if I run with gee environment
# https://stackoverflow.com/questions/75666380/attributeerror-module-ipython-utils-traitlets-has-no-attribute-unicode
# run with anaconda base environment

In [ ]:
ee.Initialize()

## Create a GEEMap Object

In [27]:
m = geemap.Map(
    center=[-5, 15], 
    zoom=3, 
    basemap = 'Esri.WorldImagery'
)
m

Map(center=[-5, 15], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

<IPython.core.display.Javascript object>

style = {'color': 'ffffff82', 'width': 1, 'lineType': 'solid', 'fillColor': 'ffffff00'}


## Add Layers to the Map

In [28]:
# add nightlights median
# https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMSLCFG
dataset_night = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
                  .filter(ee.Filter.date('2022-01-01', '2023-11-01'))
nighttime = dataset_night.select('avg_rad')
image_night = nighttime.median()
nighttimeVis = {'min': 0.0, 'max': 2.0}
m.addLayer(image_night, nighttimeVis, 'Nighttime')


In [29]:
# add WorldPop population density layer
# https://developers.google.com/earth-engine/datasets/catalog/WorldPop_GP_100m_pop#description
dataset_pop = ee.ImageCollection('WorldPop/GP/100m/pop') \
                  .filter(ee.Filter.date('2020-01-01', '2020-12-31'))
pop = dataset_pop.select('population')
image_pop = pop.median()
popVis = {
    'min': 0.0, 
    'max': 20.0,
    'palette': ['24126c', '1fff4f', 'd4ff50'],
    'opacity': 0.5
}
m.addLayer(image_pop, popVis, 'Population')


In [30]:
# overlay country boundaries with white borders
countries = ee.FeatureCollection('FAO/GAUL/2015/level0')
# styleParams = {
#   'fillColor': '00000000',
#   'color': 'ffffff',
#   'width': 1.0,
#   'lineType': 'solid'
# }
# style = {'color': 'ffffff82', 'width': 1, 'lineType': 'solid', 'fillColor': 'ffffff00'}
m.addLayer(countries, {}, 'Countries')

# note: this isn't styling the countries correctly
# manually change the country layer to have a transparent fill on the interactive map
# note do not rerun these cells multiple times, seems to freeze the map

In [31]:
# add place names
# https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0
m.add_basemap('CartoDB.DarkMatterOnlyLabels')
# m.remove_layer('CartoDB.VoyagerOnlyLabels') # doesn't work
m

Map(bottom=1352.0, center=[-5, 15], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

# Find Control Sites

Now we're looking for populated areas in the countries of interest that have nightlight values < 0.5, population values > 20 people / 100m2, and visible buildings / roads on the satellite layer. 